# User Creation for MSQL Spark Connector
MSSQL Spark connector provides an efficient write SQLServer master instance and SQL Server data pool in Big Data Clusters. To transfer data to SQL Server the MSSQL Spark Connector requires the right credentials. This notebook explains creation of the user account the permissions for use in MSSQL Spark Connector.

## What permissions does MSSQL Spark connector requires.
The answer depends on the capability that's used. MSSQL Spark connector can read, write, create tables in SQL Server Master instance and Data Pools in SQL Server 2019 Big Data Clusters. For using the full set of functionality the user account needs to have permissions to read, write, create tables and external tables in data pools.


## For non-AD cluster, run the following cells to prepare user and table:

In [6]:
-- Create a database named "connector_test_db"
IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = N'connector_test_db')
BEGIN
    Create DATABASE connector_test_db
END

-- Create a login
IF NOT EXISTS (select name from sys.server_principals where name='connector_user')
BEGIN
    CREATE LOGIN connector_user  WITH PASSWORD ='password123!#' 
END

Commands completed successfully.

Total execution time: 00:00:00.335

In [1]:
Use connector_test_db;

-- Create a database user
CREATE USER connector_user FROM LOGIN connector_user

-- To view data pool node configuration
grant VIEW DATABASE STATE to connector_user;

-- To create external tables in data pools
grant alter any external data source to connector_user;

-- To create external table
grant create table to connector_user;
grant alter any schema to connector_user;

ALTER ROLE [db_datareader] ADD MEMBER connector_user;
ALTER ROLE [db_datawriter] ADD MEMBER connector_user;

-- Create external table in data pool
CREATE EXTERNAL DATA SOURCE connector_ds  WITH (LOCATION = 'sqldatapool://controller-svc/default');
EXECUTE('USE connector_test_db; CREATE EXTERNAL TABLE [dummy3] ([number] int, [word] nvarchar(2048)) WITH (DATA_SOURCE = connector_ds, DISTRIBUTION = ROUND_ROBIN)')

-- Create a login in data pools and Provide right permissions to this user
EXECUTE( ' Use connector_test_db; CREATE LOGIN connector_user  WITH PASSWORD = ''password123!#'' ; ' )  AT  DATA_SOURCE connector_ds;
EXECUTE( ' Use connector_test_db; CREATE USER  connector_user ; ALTER ROLE [db_datareader] ADD MEMBER connector_user;  ALTER ROLE [db_datawriter] ADD MEMBER connector_user ;')  AT  DATA_SOURCE connector_ds;

Commands completed successfully.

Total execution time: 00:00:10.334

Connector also supports write when the destination table contains multiple computed columns. We will write the same data to a table that has one additional column which is computed by other columns. The table is constructed using this T-SQL. Run below cells to create the table.

In [1]:
-- Create a table with computed column net_capotal_gain
USE connector_test_db;
CREATE TABLE AdultCenses_computed_col
(
    age int,
    workclass nvarchar(max),
    fnlwgt int,
    education nvarchar(max),
    education_num int,
    marital_status nvarchar(max),
    occupation nvarchar(max),
    relationship nvarchar(max),
    race nvarchar(max),
    sex	nvarchar(max),
    capital_gain int,
    capital_loss int,
    hours_per_week int,
    native_country nvarchar(max),
    income nvarchar(max),
    net_capital_gain AS (capital_gain - capital_loss)
)

Commands completed successfully.

Total execution time: 00:00:00.157

In [2]:
-- Create a table without age column
USE connector_test_db;
CREATE TABLE AdultCenses_schema_check
(
    workclass nvarchar(max),
    fnlwgt int,
    education nvarchar(max),
    education_num int,
    marital_status nvarchar(max),
    occupation nvarchar(max),
    relationship nvarchar(max),
    race nvarchar(max),
    sex	nvarchar(max),
    capital_gain int,
    capital_loss int,
    hours_per_week int,
    native_country nvarchar(max),
    income nvarchar(max),
    net_capital_gain AS (capital_gain - capital_loss)
)

Commands completed successfully.

Total execution time: 00:00:00.156

## For AD cluster, run the following cells to prepare user and table:

In [18]:
-- Create a database named "spark_mssql_db"
IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = N'spark_mssql_db')
BEGIN
    CREATE DATABASE spark_mssql_db
END

-- Create a login from a domain user
IF NOT EXISTS (select name from sys.server_principals where name='AZDATA\testusera1')
BEGIN
    CREATE LOGIN [AZDATA\testusera1] FROM WINDOWS
END

Commands completed successfully.

Total execution time: 00:00:10.921

In [19]:
Use spark_mssql_db;

-- Create a database user
CREATE USER [AZDATA\testusera1];

-- To view data pool node configuration
GRANT VIEW DATABASE STATE to [AZDATA\testusera1];

-- To create external tables in data pools
grant alter any external data source to [AZDATA\testusera1];

-- To create external table
grant create table to [AZDATA\testusera1];
grant alter any schema to [AZDATA\testusera1];

ALTER ROLE [db_datareader] ADD MEMBER [AZDATA\testusera1];
ALTER ROLE [db_datawriter] ADD MEMBER [AZDATA\testusera1];

-- Create external table in data pool
CREATE EXTERNAL DATA SOURCE connector_ds  WITH (LOCATION = 'sqldatapool://controller-svc/default');
EXECUTE('USE spark_mssql_db; CREATE EXTERNAL TABLE [dummy3] ([number] int, [word] nvarchar(2048)) WITH (DATA_SOURCE = connector_ds, DISTRIBUTION = ROUND_ROBIN)')

-- Create a login in data pools and Provide right permissions to this user
EXECUTE( ' USE spark_mssql_db; CREATE LOGIN [AZDATA\testusera1]  FROM WINDOWS ' )  AT  DATA_SOURCE connector_ds;

EXECUTE( ' USE spark_mssql_db; CREATE USER  [AZDATA\testusera1] ; ALTER ROLE [db_datareader] ADD MEMBER [AZDATA\testusera1];  ALTER ROLE [db_datawriter] ADD MEMBER [AZDATA\testusera1] ;')  AT  DATA_SOURCE connector_ds;

Commands completed successfully.

Total execution time: 00:00:10.628